#Dataset

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d ayushmandatta1/deepdetect-2025

Dataset URL: https://www.kaggle.com/datasets/ayushmandatta1/deepdetect-2025
License(s): apache-2.0
100% 3.22G/3.23G [01:25<00:00, 189MB/s]
100% 3.23G/3.23G [01:25<00:00, 40.7MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/deepdetect-2025.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

##Removing Inconsistant Images

In [4]:
import os
from PIL import Image
import tensorflow as tf

def clean_directory(directory):
    cleaned_files_count = 0
    # Strictly enforce extensions required by TensorFlow's decode_image based on the error message
    required_image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')

    for root, _, files in os.walk(directory):
        for file in files:
            filepath = os.path.join(root, file)
            file_size = os.path.getsize(filepath)

            # First, filter out zero-byte files as they are definitely problematic
            if file_size == 0:
                print(f"Removing zero-byte file: {filepath}")
                os.remove(filepath)
                cleaned_files_count += 1
                continue

            # Check if the file has one of the strictly required image extensions
            if filepath.lower().endswith(required_image_extensions):
                try:
                    # Try to open it with PIL first for a quick check
                    Image.open(filepath).close()

                    # Then, try to decode it with TensorFlow to catch more subtle issues
                    img_bytes = tf.io.read_file(filepath)
                    _ = tf.io.decode_image(img_bytes, channels=3, expand_animations=False)

                except Exception as e:
                    # If either PIL or TensorFlow decoding fails, it's corrupted or problematic
                    print(f"Removing corrupted/undecodable image file: {filepath} - Error: {e}")
                    os.remove(filepath)
                    cleaned_files_count += 1
            else:
                # If it doesn't have a required extension, remove it.
                # This covers cases like WEBP/TIFF/PDF/etc. that TF's decoder might not support by default.
                print(f"Removing file with unsupported extension: {filepath}")
                os.remove(filepath)
                cleaned_files_count += 1
    return cleaned_files_count

print("Cleaning train directory...")
cleaned_train_count = clean_directory('/content/ddata/train')
print(f"Removed {cleaned_train_count} problematic files from train directory.")

print("Cleaning test directory...")
cleaned_test_count = clean_directory('/content/ddata/test')
print(f"Removed {cleaned_test_count} problematic files from test directory.")

Cleaning train directory...
Removed 0 problematic files from train directory.
Cleaning test directory...
Removed 0 problematic files from test directory.


In [1]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D, GlobalAveragePooling2D, Flatten
from keras.models import Model

##Implementing Image Batches & Normalizing


In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/ddata/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 128,
    color_mode = 'rgb',
    image_size = (240, 240)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/ddata/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 64,
    color_mode = 'rgb',
    image_size = (240, 240)
)

Found 90407 files belonging to 2 classes.
Found 21776 files belonging to 2 classes.


In [3]:
from keras.applications.efficientnet import preprocess_input
def process(image, label):
    image = preprocess_input(image)
    return image,label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

#Phase 1

In [4]:
from keras.applications import EfficientNetB1

In [6]:
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomContrast(0.2)
])

In [7]:
inputs = keras.Input(shape=(240,240,3))

x = data_augmentation(inputs)
x = preprocess_input(x)   # EfficientNet preprocessing

base_model = EfficientNetB1(include_top=False, weights='imagenet')
base_model.trainable = False    # IMPORTANT (Phase 1)

x = base_model(x, training=False)

x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

outputs = Dense(2, activation='softmax')(x)

model = keras.Model(inputs, outputs)

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
history1 = model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
707/707 ━━━━━━━━━━━━━━━━━━━━ 392s 526ms/step - accuracy: 0.6277 - loss: 0.8403 - val_accuracy: 0.6762 - val_loss: 0.6418
Epoch 2/5
707/707 ━━━━━━━━━━━━━━━━━━━━ 352s 498ms/step - accuracy: 0.7247 - loss: 0.5839 - val_accuracy: 0.6985 - val_loss: 0.6089
Epoch 3/5
707/707 ━━━━━━━━━━━━━━━━━━━━ 352s 497ms/step - accuracy: 0.7522 - loss: 0.5182 - val_accuracy: 0.7367 - val_loss: 0.5358
Epoch 4/5
707/707 ━━━━━━━━━━━━━━━━━━━━ 352s 497ms/step - accuracy: 0.7747 - loss: 0.4793 - val_accuracy: 0.7319 - val_loss: 0.5451
Epoch 5/5
707/707 ━━━━━━━━━━━━━━━━━━━━ 352s 497ms/step - accuracy: 0.7809 - loss: 0.4595 - val_accuracy: 0.7512 - val_loss: 0.5155


#2nd Phase

In [10]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
history2 = model.fit(
    train_ds,
    validation_data=test_ds
    epochs=10,
    callbacks=[
        keras.callbacks.EarlyStopping(
            patience=3,
            restore_best_weights=True
        )
    ]
)


Epoch 1/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 431s 583ms/step - accuracy: 0.7323 - loss: 0.5463 - val_accuracy: 0.7696 - val_loss: 0.4875
Epoch 2/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 394s 557ms/step - accuracy: 0.7779 - loss: 0.4721 - val_accuracy: 0.7900 - val_loss: 0.4525
Epoch 3/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 394s 557ms/step - accuracy: 0.8039 - loss: 0.4206 - val_accuracy: 0.8075 - val_loss: 0.4261
Epoch 4/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 394s 557ms/step - accuracy: 0.8194 - loss: 0.3962 - val_accuracy: 0.8248 - val_loss: 0.3933
Epoch 5/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 394s 557ms/step - accuracy: 0.8319 - loss: 0.3743 - val_accuracy: 0.8330 - val_loss: 0.3792
Epoch 6/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 409s 578ms/step - accuracy: 0.8404 - loss: 0.3570 - val_accuracy: 0.8356 - val_loss: 0.3722
Epoch 7/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 408s 577ms/step - accuracy: 0.8515 - loss: 0.3385 - val_accuracy: 0.8442 - val_loss: 0.3551
Epoch 8/10
707/707 ━━━━━━━━━━━━━━━━━━━━ 427s 556ms/step - accuracy: 0.8572 -

#Performance Visualization

to be continued ...